# Adding Command-Line Arguments

In [37]:
import sys
import argparse
import json

### Warning!

Using 'args = parser.parse_args()' will throw an exception:

"usage: ipykernel_launcher.py [-h] [--location LOCATION] [--download]
                             [--itag ITAG]
ipykernel_launcher.py: error: unrecognized arguments: -f"

Because when running the script as an IPython kernel, additional arguments are passed automatically, such as -f to specify the kernel file. These arguments are not recognized by your script, resulting in the error.

Changing this line with 'args, _ = parser.parse_known_args()' will ignore to unknown args.

In [39]:
parser = argparse.ArgumentParser()

parser.add_argument('--videourl', help='Youtube Video url to be downloaded')
parser.add_argument('--location', help='Location to write the Youtube video file')
parser.add_argument('--download', action='store_true', help='Enable Video&Audio Download')
parser.add_argument('--itag', type=int, default=-1, help='Stream ID to spesify the quality or type of video. If empty, the highest resolution video will be downloaded')

args, _ = parser.parse_known_args()

videourl = args.videourl
location = args.location
download = args.download
itag = args.itag

# Pytube Configs And Video Downloading

In [1]:
from pytube import YouTube
import pytube
import warnings

pytube.innertube._default_clients['ANDROID']=pytube.innertube._default_clients['WEB']

In [2]:
yt = None
try:
    yt = YouTube(videourl)
except YouTube.RegexMatchError:
    warnings.warn("Invalid YouTube video link!")
    raise RuntimeError("Invalid YouTube video link!") from None
    
#link = "https://www.youtube.com/watch?v=NFtQOxjWfaA"
#yt = YouTube(link)

In [3]:
print("Title: ",yt.title)
print("Number of views: ",yt.views)
print("Length of video: ",yt.length,"seconds")
print("Description: ",yt.description)
print("Ratings: ",yt.rating)

Title:  Anatomy of a Headshot
Number of views:  23867816
Length of video:  415 seconds
Description:  None
Ratings:  None


In [6]:
for stream in yt.streams.filter(only_audio=True):
    print(stream)

<Stream: itag="139" mime_type="audio/mp4" abr="48kbps" acodec="mp4a.40.5" progressive="False" type="audio">
<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
<Stream: itag="249" mime_type="audio/webm" abr="50kbps" acodec="opus" progressive="False" type="audio">
<Stream: itag="250" mime_type="audio/webm" abr="70kbps" acodec="opus" progressive="False" type="audio">
<Stream: itag="251" mime_type="audio/webm" abr="160kbps" acodec="opus" progressive="False" type="audio">


In [7]:
for stream in yt.streams.filter(only_video=True):
    print(stream)

<Stream: itag="137" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="avc1.640028" progressive="False" type="video">
<Stream: itag="248" mime_type="video/webm" res="1080p" fps="30fps" vcodec="vp9" progressive="False" type="video">
<Stream: itag="136" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">
<Stream: itag="247" mime_type="video/webm" res="720p" fps="30fps" vcodec="vp9" progressive="False" type="video">
<Stream: itag="135" mime_type="video/mp4" res="480p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">
<Stream: itag="244" mime_type="video/webm" res="480p" fps="30fps" vcodec="vp9" progressive="False" type="video">
<Stream: itag="134" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.4d401e" progressive="False" type="video">
<Stream: itag="243" mime_type="video/webm" res="360p" fps="30fps" vcodec="vp9" progressive="False" type="video">
<Stream: itag="133" mime_type="video/mp4" res="240p" fps="30fps" v

In [8]:
for stream in yt.streams.filter(progressive=True):
    print(stream)

<Stream: itag="17" mime_type="video/3gpp" res="144p" fps="7fps" vcodec="mp4v.20.3" acodec="mp4a.40.2" progressive="True" type="video">
<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">
<Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">


In [9]:
ys = None
if (itag == -1):
    ys = yt.streams.get_highest_resolution
else:
    ys = yt.streams.get_by_itag('250')

In [11]:
save_location = "C:/users/emirs/Desktop/"
if location is not None:
    save_location = location

In [12]:
if download:
    ys.download(location)

'C:/users/emirs/Desktop/Anatomy of a Headshot.webm'

# Convert Video Streaming Options To A JSON File

In [25]:
import json

In [34]:
only_audio_Streamings = list(yt.streams.filter(only_audio=True))
only_video_Streamings = list(yt.streams.filter(only_video=True))
progressive_Streamings = list(yt.streams.filter(progressive=True))

# Extract necessary attributes for each stream
only_audio_data = []
for stream in only_audio_Streamings:
    only_audio_data.append({
        'url': stream.url,
        'itag': stream.itag,
        'mime_type': stream.mime_type,
        # Add other required attributes here
    })

only_video_data = []
for stream in only_video_Streamings:
    only_video_data.append({
        'url': stream.url,
        'itag': stream.itag,
        'mime_type': stream.mime_type,
        # Add other required attributes here
    })

progressive_data = []
for stream in progressive_Streamings:
    progressive_data.append({
        'url': stream.url,
        'itag': stream.itag,
        'mime_type': stream.mime_type,
        # Add other required attributes here
    })

# Create a dictionary with the extracted data
data = {
    'only_audio_Streamings': only_audio_data,
    'only_video_Streamings': only_video_data,
    'progressive_Streamings': progressive_data
}

In [35]:
json_data = json.dumps(data, indent=4)

In [36]:
with open('data.json', 'w') as file:
    file.write(json_data)